In [ ]:
import os
import numpy as np
import pydicom

In [ ]:
# Root directory of all scans, it contains folders like LUNG1-001_20180209_CT
INPUT_ROOT_DIR = "./data_dcm"
# Output folder, .npy files will be placed here
DATA_OUTPUT_DIR = "./data_npy"

In [ ]:
# List all dirs in selected root
_, dirs, _ = next(os.walk(f"{INPUT_ROOT_DIR}"))
for cur_dir in dirs:
    cur_dir_full = f"{INPUT_ROOT_DIR}/{cur_dir}"
    # List all files in current dir
    _, _, files = next(os.walk(cur_dir_full))
    # Take only .dcm files
    files = filter(lambda f: f.split(".")[1] == "dcm", files)
    # Sort by file number
    dicoms = sorted(files, key=lambda f: int(f.split(".")[0]))
    np_list = []
    for dicom in dicoms:
        dicom = pydicom.read_file(f"{cur_dir_full}/" + dicom)
        # Check whether the file contains image data we're interested in
        if dicom[(0x0008, 0x0016)].repval == "CT Image Storage":
            np_pixel_array = dicom.pixel_array
            np_list.append(np_pixel_array)
    # Write as NumPy array
    if np_list:
        npy = np.array(np_list)
        os.makedirs(DATA_OUTPUT_DIR, exist_ok=True)
        np.save(f"{DATA_OUTPUT_DIR}/{cur_dir}.npy", npy)